# Actividad 6 - Secuenciación de Tareas 

### Genetic Algorithm

- Luis Maximiliano López Ramírez Matricula: A08833321
- Annete Pamela Ruiz Abreu Matricula: A01423595
-  Andrés Navarro Matricula: A00833287
-  Andrea Axel Hernández Galgani Matricula: A00835225
- Jorge Raúl Rocha López Matricula: A01740816

Instrucciones: Resuelve completa y correctamente cada uno de los siguientes puntos. Se
evalúa el procedimiento. Resultado sin procedimiento no tiene puntaje alguno.
Dado un conjunto M = {1, 2, · · · , m} de tareas que se van a secuenciar en una sola máquina,
cada tarea i ∈ M tiene un tiempo de procesamiento ti y una fecha límite di para ser realizada. Si
la tarea i se completa antes de la fecha límite, incurre en un costo de retenci´on hi por
unidad de tiempo. Una tarea retardada i da como resultado un costo de penalización ci por
unidad de tiempo.



\[
\begin{array}{|c|c|c|c|c|c|}
\hline
\text{Tarea} & t_i & d_i & h_i & c_i \\
\hline
1 & 10 & 15 & 3 & 10 \\
2 & 8  & 20 & 2 & 22 \\
3 & 6  & 10 & 5 & 10 \\
4 & 7  & 30 & 4 & 8  \\
5 & 4  & 12 & 6 & 15 \\
\hline
\end{array}
\]


2. Diseña un algoritmo genético para determinar la mejor secuencia de tareas S tal que
los costos por retención y penalización sean minimizados.


In [ ]:
import random
import time


tareas_tiempo = {1:(10,15,3,10),
                 2:(8,20,2,22),
                  3:(6,10,5,10),
                   4:(7,30,4,8),
                    5:(4,12,6,15)}

tareas_letras = {'A':1,
                 'B':2,
                 'C':3,
                 'D':4,
                 'E':5}



tareas =[]
for key, tarea in tareas_letras.items():
    tareas.append(key)



class Asignacion_algoritmo_gen:
    def __init__(self, punto_cruce, prob_mutation, epochs):
        self.generacion = [[] for u in range(10)]
        self.punto_cruce = punto_cruce
        self.prob_mutation = prob_mutation
        self.epochs = epochs
        for u in range(len(self.generacion)):
            tareas_c = tareas[:]

            for j in range(len(tareas_letras)):
                x = random.choice(tareas_c)
                self.generacion[u].append(x)

                tareas_c.remove(x)
            self.generacion[u] = ''.join(self.generacion[u])
    def Calcular_costo(self):
        self.tareas_num = [[] for u in range(len(self.generacion))]
        for u in range(len(self.generacion)):
            for j in range(len(self.generacion[u])):
                self.tareas_num[u].append((tareas_letras[self.generacion[u][j]]))

        self.costo_generacion = [0 for u in range(len(self.generacion))]
        for u in range(len(self.tareas_num)):
            dia_actual = 0
            for j in range(len(self.tareas_num[u])):
                tupla = tareas_tiempo[self.tareas_num[u][j]]
                dias_tarda = tupla[0]
                dia_lim = tupla[1]
                pen_antes = tupla[2]
                pen_despues = tupla[3]

                dia_actual += dias_tarda

                if(dia_actual > dia_lim):
                    self.costo_generacion[u] += pen_despues* (dia_actual- dia_lim)
                elif(dia_actual < dia_lim):
                    self.costo_generacion[u] += pen_antes * (dia_lim - dia_actual)

        self.fitness = [1/self.costo_generacion[u] for u in range(len(self.costo_generacion))]

    def Seleccionar_padres(self):
        self.prob = [(self.fitness[u]/ sum(self.fitness)) for u in range(len(self.fitness))]
        self.prob_li = []
        self.prob_ls = []
        acum_0 = 0
        acum_1 = 0
        self.padres = []

        for u in range(len(self.fitness)):
            acum_1 += self.prob[u]
            self.prob_ls.append(acum_1)
            self.prob_li.append(acum_0)
            acum_0 += self.prob[u]

        while True:
            x = random.random()

            for u in range(len(self.prob)):
                if((x >= self.prob_li[u]) and (x < self.prob_ls[u]) and (u not in self.padres)):
                    self.padres.append(u)
                    break

            if(len(self.padres) == 2):
                break
    def Reproducir(self):
        self.gen_padres = [self.generacion[self.padres[0]], self.generacion[self.padres[1]]]
        self.hijos = []
        madre = self.gen_padres[0]
        padre = self.gen_padres[1]

        hijos_1_pos =(madre[:self.punto_cruce]+padre[self.punto_cruce:])
        hijos_1_pos = list(hijos_1_pos)
        hijos_2_pos = (padre[:self.punto_cruce]+madre[self.punto_cruce:])
        hijos_2_pos = list(hijos_2_pos)


        # Hacer factible al hijo 1:
        letras_a_cambiar_hijo_1 = []

        for u in range(self.punto_cruce, len(hijos_1_pos)):
            letras_que_no = list(hijos_1_pos[:u])

            for j in range(len(padre)):
                if(padre[j] not in letras_que_no):
                    letras_a_cambiar_hijo_1.append(padre[j])

            break


        for u in range(self.punto_cruce, len(hijos_1_pos)):
            hijos_1_pos[u] = letras_a_cambiar_hijo_1[u-self.punto_cruce]

        hijos_1_pos = ''.join(hijos_1_pos)
        self.hijos.append(hijos_1_pos)

        # Hacer factible al hijo 2:
        letras_a_cambiar_hijo_2 = []
        for u in range(self.punto_cruce, len(hijos_2_pos)):
            letras_que_no = list(hijos_2_pos[:u])

            for j in range(len(madre)):
                if(madre[j] not in letras_que_no):
                    letras_a_cambiar_hijo_2.append(madre[j])

            break

        for u in range(self.punto_cruce, len(hijos_2_pos)):
            hijos_2_pos[u] = letras_a_cambiar_hijo_2[u-self.punto_cruce]

        hijos_2_pos = ''.join(hijos_2_pos)
        self.hijos.append(hijos_2_pos)


    def Mutar_hijos(self):
        for u in range(len(self.hijos)):
            x = random.random()
            if(x <= self.prob_mutation):
                hijo_mutado = list(self.hijos[u])
                indices = [u for u in range(len(hijo_mutado))]
                i1 = random.choice(indices)
                indices.remove(i1)
                i2 = random.choice(indices)

                letra_1 = hijo_mutado[i1]
                letra_2 = hijo_mutado[i2]

                hijo_mutado[i1] = letra_2
                hijo_mutado[i2] = letra_1
                self.hijos[u] = ''.join(hijo_mutado)

    def Calcular_costo_hijos(self):
        self.tareas_num_hijos = [[] for u in range(len(self.hijos))]
        for u in range (len(self.hijos)):
            for j in range(len(self.hijos[u])):
                self.tareas_num_hijos[u].append(tareas_letras[self.hijos[u][j]])


        self.costo_hijos = [0 for u in range(len(self.hijos))]


        for u in range(len(self.tareas_num_hijos)):
            dia_actual = 0
            for j in range(len(self.tareas_num_hijos[u])):
                tupla = tareas_tiempo[self.tareas_num_hijos[u][j]]
                dias_tarda = tupla[0]
                dia_lim = tupla[1]
                pen_antes = tupla[2]
                pen_despues = tupla[3]

                dia_actual += dias_tarda

                if(dia_actual > dia_lim):
                    self.costo_hijos[u] += pen_despues* (dia_actual- dia_lim)
                elif(dia_actual < dia_lim):
                    self.costo_hijos[u] += pen_antes * (dia_lim - dia_actual)

        self.fitness_hijos = [1/self.costo_hijos[u] for u in range(len(self.costo_hijos))]

    def Reemplazar_pob(self):
        self.dic_costo_gen = {index: element for index, element in enumerate(self.costo_generacion)}
        self.costo_gen_ordenado = sorted(self.dic_costo_gen.items(), key=lambda element: element[1], reverse=True)

        indice_min_1, costo_maximo_1 = self.costo_gen_ordenado[0]
        indice_min_2, costo_maximo_2 = self.costo_gen_ordenado[1]

        self.dic_costo_hijos = {index: element for index, element in enumerate(self.costo_hijos)}
        self.costo_hijos_ordenado = sorted(self.dic_costo_hijos.items(), key=lambda elemento: elemento[1], reverse=True)


        indice_min_hijos, costo_maximo_hijos_1 = self.costo_hijos_ordenado[0]
        indice_min_hijos2, costo_maximo_hijos_2 = self.costo_hijos_ordenado[1]


        if(costo_maximo_1 > costo_maximo_hijos_1):
            self.generacion[indice_min_1] = self.hijos[indice_min_hijos]
        if(costo_maximo_2 > costo_maximo_hijos_2):
            self.generacion[indice_min_2] = self.hijos[indice_min_hijos2]





    def Mejor_Poblacion(self):

        mejor_indice = self.costo_generacion.index(min(self.costo_generacion))

        mejor_cromosoma = self.generacion[mejor_indice]
        mejor_costo = self.costo_generacion[mejor_indice]
        mejor_fitness = self.fitness[mejor_indice]

        return mejor_cromosoma, mejor_costo, mejor_fitness


    def Algoritmo_genetico(self):
        for u in range(self.epochs):
            self.Calcular_costo()
            self.Seleccionar_padres()
            self.Reproducir()
            self.Mutar_hijos()
            self.Calcular_costo_hijos()
            self.Reemplazar_pob()


            mejor_cromosoma, mejor_costo, mejor_fitness = self.Mejor_Poblacion()

            print(f'Época: {u}  | Mejor asignación: {mejor_cromosoma}  | Mejor costo: {mejor_costo} | Mejor fitness: {mejor_fitness}')



tiempo_inicio = time.time()
a = Asignacion_algoritmo_gen(3,0.05,500)
a.Algoritmo_genetico()
tiempo_final = time.time()
print(f'El tiempo que se trado el programa en ejecutarse es: {tiempo_final-tiempo_inicio}')

Época: 0  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 1  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 2  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 3  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 4  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 5  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 6  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 7  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 8  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 9  | Mejor asignación: ECBAD  | Mejor costo: 222 | Mejor fitness: 0.0045045045045045045
Época: 10  | Mejor asignación: ECBAD  | Mejor cost